In [ ]:
import numpy 
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("/content/sample_data/zomato.csv",encoding='latin-1')

In [ ]:

df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3.0,4.8,Dark Green,Excellent,314.0
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3.0,4.5,Dark Green,Excellent,591.0
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4.0,4.4,Green,Very Good,270.0
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4.0,4.9,Dark Green,Excellent,365.0
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4.0,4.8,Dark Green,Excellent,229.0


In [ ]:
df.shape

(4394, 21)

In [ ]:
df["City"].value_counts()

New Delhi         1834
Gurgaon           1118
Faridabad          251
Ghaziabad           25
Bhubaneshwar        21
                  ... 
Weirton              1
Winchester Bay       1
Yorkton              1
Flaxton              1
Fernley              1
Name: City, Length: 110, dtype: int64

In [ ]:
df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    1
Price range             1
Aggregate rating        1
Rating color            1
Rating text             1
Votes                   1
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
#df["Cuisines"].replace(np.nan,df["Cuisines"].mean(),inplace=True)

In [ ]:
df.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                0
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [ ]:
df_nd = df[df["City"]=="New Delhi"]

In [ ]:
df_nd.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
2560,18287358,Food Cloud,1,New Delhi,"Aaya Nagar, New Delhi",Aaya Nagar,"Aaya Nagar, New Delhi",0.000000,0.000000,Cuisine Varies,...,Indian Rupees(Rs.),No,No,No,No,2.0,0.0,White,Not rated,2.0
2561,18216944,Burger.in,1,New Delhi,"84, Near Honda Showroom, Adchini, New Delhi",Adchini,"Adchini, New Delhi",77.196923,28.535382,Fast Food,...,Indian Rupees(Rs.),No,Yes,No,No,1.0,3.2,Orange,Average,46.0
2562,313333,Days of the Raj,1,New Delhi,"81/3, 1st Floor, Qutub Residency, Adchini, New...",Adchini,"Adchini, New Delhi",77.197475,28.535493,"North Indian, Seafood, Continental",...,Indian Rupees(Rs.),Yes,Yes,No,No,3.0,3.4,Orange,Average,45.0
2563,18384127,Dilli Ka Dhaba,1,New Delhi,"66 A, Ground Floor, Sri Aurobindo Marg, Adchin...",Adchini,"Adchini, New Delhi",77.198033,28.537547,"South Indian, North Indian",...,Indian Rupees(Rs.),No,No,No,No,2.0,2.6,Orange,Average,11.0
2564,582,Govardhan,1,New Delhi,"84, Adjacent Hero Motor Bike Showroom, Main Me...",Adchini,"Adchini, New Delhi",77.196924,28.535523,"South Indian, North Indian, Chinese",...,Indian Rupees(Rs.),No,Yes,No,No,2.0,3.4,Orange,Average,238.0


In [ ]:
df_nd.shape

(1833, 21)

In [ ]:
df_nd["Locality"].value_counts()

Connaught Place              122
Defence Colony                86
Hauz Khas                     74
Chandni Chowk                 67
Greater Kailash (GK) 1        66
                            ... 
Holiday Inn, Aerocity          1
Dr. Zakir Hussain Marg         1
Durga Puri                     1
Gourmet Hub, Pashim Vihar      1
Aaya Nagar                     1
Name: Locality, Length: 102, dtype: int64

In [ ]:
df_cp = df_nd[df_nd["Locality"]=="Connaught Place"]

In [ ]:
df_cp.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
2999,53,Amber,1,New Delhi,"N-19, Connaught Place, New Delhi",Connaught Place,"Connaught Place, New Delhi",77.220891,28.630197,"North Indian, Chinese, Mughlai",...,Indian Rupees(Rs.),Yes,Yes,No,No,3.0,2.6,Orange,Average,152.0
3000,307271,Attitude Kitchen & Bar,1,New Delhi,"N-12, Outer Circle, Connaught Place, New Delhi",Connaught Place,"Connaught Place, New Delhi",77.220352,28.630044,"North Indian, Continental, Italian",...,Indian Rupees(Rs.),Yes,No,No,No,3.0,2.9,Orange,Average,140.0
3001,596,Cafe Coffee Day,1,New Delhi,"4, Rajiv Chowk, Connaught Place, New Delhi",Connaught Place,"Connaught Place, New Delhi",77.219813,28.632961,Cafe,...,Indian Rupees(Rs.),No,Yes,No,No,1.0,3.4,Orange,Average,277.0
3002,1127,Castle 9,1,New Delhi,"B-45/47, 1st Floor, Near PVR Plaza, Connaught ...",Connaught Place,"Connaught Place, New Delhi",77.219498,28.634857,"Finger Food, Continental, North Indian, Chinese",...,Indian Rupees(Rs.),Yes,No,No,No,3.0,3.1,Orange,Average,1099.0
3003,4237,Costa Coffee,1,New Delhi,"Shop 49, Bengali Market, Connaught Place, New ...",Connaught Place,"Connaught Place, New Delhi",77.232192,28.629256,Cafe,...,Indian Rupees(Rs.),No,No,No,No,2.0,3.4,Orange,Average,76.0


In [ ]:
df_cp.reset_index(inplace=True)

In [ ]:
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3.0,4.8,Dark Green,Excellent,314.0
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3.0,4.5,Dark Green,Excellent,591.0
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4.0,4.4,Green,Very Good,270.0
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4.0,4.9,Dark Green,Excellent,365.0
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4.0,4.8,Dark Green,Excellent,229.0


In [ ]:
def rest_recoma_fun(location,title):

  global data_sample
  global cosine_sim
  global tfidf_matrix
  global feature
  global idx


  data_sample = df_nd[df_nd["Locality"] == location]
  data_sample.reset_index(inplace=True)

  #Features Extraction 

  data_sample["Split"] = "X"

  for i in range(len(data_sample)):
    #split_data = re.split(r'[,]',data_sample["Cuisines"][i])
    split_data = data_sample["Cuisines"][i].split(",") #

    # for k,l in enumerate(split_data):
    #   split_data[k] = (split_data[k].replace(" ",""))

    for j in range(len(split_data)):
      split_data[j] = (split_data[j].replace(" ","")) #["abc ","bcd "]

    split_data = " ".join(split_data[:])
    data_sample["Split"].iloc[i] = split_data


  #TF-IDF 
  tfidf = TfidfVectorizer(stop_words="english")

  #Applying TfDIF
  tfidf_matrix = tfidf.fit_transform(data_sample["Split"])


  #print(tfidf.get_feature_names())

  cosine_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)

  # Columns names are uing for index
  corpus_indexx = [n for n in data_sample["Split"]]

  indices = pd.Series(data_sample.index,index = data_sample["Restaurant Name"]).drop_duplicates()

  idx = indices[title]

  #print(cosine_sim[idx])

  #pairwise 
  #[a,v,b,d] - 1
  #[d,,c,f,f] - 0.8
  #[m,d,f,f,x] - 0.5

  sim_score = []
  for i,j in enumerate(cosine_sim[idx]):
    k = data_sample["Aggregate rating"].iloc[i]

    if j !=0:
      sim_score.append((i,j,k))
      

  #sorting the resturant name on based on rating
  sim_score = sorted(sim_score,key = lambda x : (x[1],x[2]),reverse = True)

  #top 5 similarity cuisones
  sim_score = sim_score[:100]


  rest_idx = [i[0] for i in sim_score]

  data_x = data_sample[["Restaurant Name","Cuisines","Aggregate rating"]].iloc[rest_idx]
  

  data_x["Cosine Simi"] = 0

  for i,j in enumerate(sim_score):
    data_x["Cosine Simi"].iloc[i] = round(sim_score[i][1],2)


  return data_x[["Restaurant Name","Aggregate rating"]][1:].reset_index().iloc[:,1:]



In [ ]:
rest_recoma_fun("Connaught Place","Costa Coffee")

,Restaurant Name,Aggregate rating
0,Cha Bar,3.9
1,Starbucks,3.8
2,Cafe Coffee Day The Square,3.6
3,Cafe Coffee Day,3.4
4,Costa Coffee,3.4
5,Caffe Tonino,3.9
6,Chaayos,3.9
7,Life Caffe,3.6
